In [1]:
import tensorflow as tf
x=tf.random.normal([2,784])
w1=tf.Variable(tf.random.truncated_normal([784,256],stddev=0.1))
b1=tf.Variable(tf.zeros([256]))
o1=tf.matmul(x,w1)+b1
o1=tf.nn.relu(o1)

In [2]:
o1

<tf.Tensor: id=32, shape=(2, 256), dtype=float32, numpy=
array([[2.4008474e+00, 0.0000000e+00, 1.0533967e+00, 1.9050387e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        2.4285345e+00, 0.0000000e+00, 4.9165887e-01, 6.4184964e-01,
        0.0000000e+00, 4.4998617e+00, 0.0000000e+00, 2.5313141e+00,
        0.0000000e+00, 0.0000000e+00, 7.2445500e-01, 1.9040405e+00,
        0.0000000e+00, 8.1945801e-01, 3.1571443e+00, 0.0000000e+00,
        0.0000000e+00, 1.3974783e+00, 2.1436937e+00, 0.0000000e+00,
        1.8838941e+00, 1.1058155e+00, 0.0000000e+00, 4.5909411e-01,
        4.1159444e+00, 0.0000000e+00, 8.8439834e-01, 5.6431584e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 2.9641042e+00, 2.4850104e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 2.9998331e+00,
        2.8214192e+00, 0.0000000e+00, 7.1735066e-01, 5.7817742e-02,
        0.0000000e+00, 0.0000000e+00, 1.9933025e+00, 2.8085

In [3]:
x=tf.random.normal([4,28*28])
from tensorflow.keras import layers
fc=layers.Dense(512,activation=tf.nn.relu)
h1=fc(x)
import keras

Using TensorFlow backend.


In [4]:
import pandas as pd

# 在线下载汽车效能数据集
dataset_path = keras.utils.get_file("auto-mpg.data", "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")
#利用pandas读取数据集，字段有效能（公里数每加仑），气缸数，排量，马力，重量# 加速度，型号年份，产地
column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight','Acceleration', 'Model Year', 'Origin']
raw_dataset = pd.read_csv(dataset_path, names=column_names,na_values = "?", comment='\t',sep=" ", skipinitialspace=True)
dataset = raw_dataset.copy()
# 查看部分数据dataset.head()原始数据中的数据可能含有空字段(缺失值)的数据项，需要清除这些记录项：
dataset.isna().sum()# 统计空白数据
dataset = dataset.dropna()# 删除空白数据项
dataset.isna().sum()# 再次统计空白数据清除后，数据集记录项减为392项。由于Origin字段为类别数据，我们将其移动出来，并转换为新的3个字段：USA，Europe和Japan，分别代表是否来自此产地：# 处理类别型数据，其中origin列代表了类别1,2,3,分布代表产地：美国、欧洲、日本# 先弹出(删除并返回)origin这一列
origin = dataset.pop('Origin')# 根据origin列来写入新的3个列
dataset['USA'] = (origin == 1)*1.0
dataset['Europe'] = (origin == 2)*1.0
dataset['Japan'] = (origin == 3)*1.0
dataset.tail()#按着8:2的比例切分训练集和测试集：# 切分为训练集和测试集
train_dataset = dataset.sample(frac=0.8,random_state=0)


In [5]:
test_dataset = dataset.drop(train_dataset.index) 
#将MPG字段移出为标签数据：# 移动MPG油耗效能这一列为真实标签
Ytrain_labels = train_dataset.pop('MPG')
test_labels = test_dataset.pop('MPG')#统计训练集的各个字段数值的均值和标准差，并完成数据的标准化：# 查看训练集的输入X的统计数据
train_stats = train_dataset.describe()
train_stats.pop("MPG")
train_stats = train_stats.transpose()
# 标准化数据
def norm(x):
    return(x -train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)#打印出训练集和测试集的大小：
print(normed_train_data.shape,train_labels.shape)
print(normed_test_data.shape, test_labels.shape)#(314, 9) (314,)# 训练集共314行，输入特征长度为9,标签用一个标量表示(78, 9) (78,)# 测试集共78行，输入特征长度为9,标签用一个标量表示利用切分的训练集数据构建数据集对象：
train_db = tf.data.Dataset.from_tensor_slices((normed_train_data.values, train_labels.values))# 构建Dataset对象
train_db = train_db.shuffle(100).batch(32)# 随机打散，批量化

KeyError: 'MPG'

In [ ]:
class Network(keras.Model):
    def __init__(self):
        super(Network,self).__init__()
        self.fc1=layers.Dense(64,activaion='relu')
        self.fc2=layers.Dense(64,activation='relu')
        self.fc3=layers.Dense(1)
    def call(self,inputs,training=None,mask=None):
        x=self.fc1(inputs)
        x=self.fc2(x)
        x=self.fc3(x)
        
        return x

In [ ]:
model=Network()
model.build(input_shape=(4,9))
model.summary()
optimizer=tf.keras.optimizers.RMSprop(0.001)

In [7]:
for epoch in range(200):
    for step,(x,y) in enumerate(train_db):
        with tf.GradientTape() as tape:
            out=model(x)
            loss=tf.reduce_mean(losses.MSE(y,out))
            mae_loss=tf.reduce_mean(losses.MAE(y,out))
            if step %10==0:
                print(epoch,step,float(loss))
            grads=tape.gradient(loss,model.trainable_variable)
            optimizer.apply_gradients(zip(grads),model.trainable_variables)

NameError: name 'train_db' is not defined